In [137]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
rating = pd.read_csv('rating.csv')
anime = pd.read_csv('anime.csv')
#number of users
n_users = rating.user_id.unique().shape[0]
# drop rows with rating -1
#rating = rating[rating.rating != -1]

genres = set()
for i in anime['genre']:
    if type(i) != float:
        for j in i.split(', '):
            genres.add(j)
genres = list(genres)
anime = anime.drop(['type', 'episodes', 'rating', 'members'], axis=1)
# add generes list as columns in anime dataframe
ad = pd.DataFrame(columns=genres)
anime = pd.concat([anime, ad], axis=1)
# add 1 if anime has that genre else 0
for i in range(len(anime)):
    if type(anime['genre'][i]) != float:
        for j in anime['genre'][i].split(', '):
            anime[j][i] = 1
# drop unnecessary columns
anime = anime.drop(['genre'], axis=1)
# fillna
anime = anime.fillna(0)
print(anime.head())

<ipython-input-137-c3ad5002fd2f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime[j][i] = 1


   anime_id                              name  Mystery  Ecchi  Shounen  \
0     32281                    Kimi no Na wa.        0      0        0   
1      5114  Fullmetal Alchemist: Brotherhood        0      0        1   
2     28977                          Gintama°        0      0        1   
3      9253                       Steins;Gate        0      0        0   
4      9969                     Gintama&#039;        0      0        1   

   Adventure  Sci-Fi  Sports  Yuri  Historical  ...  Cars  Vampire  Romance  \
0          0       0       0     0           0  ...     0        0        1   
1          1       0       0     0           0  ...     0        0        0   
2          0       1       0     0           1  ...     0        0        0   
3          0       1       0     0           0  ...     0        0        0   
4          0       1       0     0           1  ...     0        0        0   

   Josei  Harem  Martial Arts  Shoujo  Psychological  Slice of Life  \
0      0 

In [138]:
merged = pd.merge(rating, anime, on='anime_id')
merged = merged.drop(['name'], axis=1)
merged


,user_id,anime_id,rating,Mystery,Ecchi,Shounen,Adventure,Sci-Fi,Sports,Yuri,...,Cars,Vampire,Romance,Josei,Harem,Martial Arts,Shoujo,Psychological,Slice of Life,Supernatural
0,1,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,3,20,8.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5,20,6.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,6,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,10,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315530,3219,6297,9.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315531,3219,10202,9.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315532,3231,2662,10.0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315533,3231,10549,-1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
from sklearn.preprocessing import LabelEncoder
user_enc = LabelEncoder()
merged['user_id'] = user_enc.fit_transform(merged['user_id'].values)
n_users = merged['user_id'].nunique()

item_enc = LabelEncoder()
merged['anime_id'] = item_enc.fit_transform(merged['anime_id'].values)
n_movies = merged['anime_id'].nunique()


n_users, n_movies

(2462, 6175)

In [139]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import copy

# Split the dataset into features and target
X = merged.drop(['anime_id','rating'],axis=1)
X = X.fillna(0)

hold = copy.deepcopy(merged)
scaler = MinMaxScaler()
merged['rating'] = scaler.fit_transform(merged['rating'].values.reshape(-1,1))

y = merged['rating']
y = y.replace(-1,0)
y = y.fillna(0)


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
hold


,user_id,anime_id,rating,Mystery,Ecchi,Shounen,Adventure,Sci-Fi,Sports,Yuri,...,Cars,Vampire,Romance,Josei,Harem,Martial Arts,Shoujo,Psychological,Slice of Life,Supernatural
0,1,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,3,20,8.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,5,20,6.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,6,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,10,20,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315530,3219,6297,9.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315531,3219,10202,9.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315532,3231,2662,10.0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315533,3231,10549,-1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [125]:

n_factors = 42
X_train_array = [X_train['user_id'].values, X_train.iloc[:, 2:].values]
X_test_array = [X_test['user_id'].values,X_test.iloc[:, 2:].values]




In [126]:
from keras.layers import Input, Reshape, Dot, Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Concatenate,Flatten
from keras.models import Model

def Recommend(n_users,n_factors):

  user = Input(shape=(1,))
  genre = Input(shape=(42,))


  u = Embedding(n_users, n_factors, embeddings_initializer='he_normal')(user)
  u = Reshape((n_factors,))(u)

  concat = Concatenate()([u, genre])

  dense1 = Dense(units=256, activation='relu')(concat)
  dropout1 = Dropout(0.5)(dense1)

  dense2 = Dense(units=128, activation='relu')(dropout1)
  dropout2 = Dropout(0.5)(dense2)

  dense3 = Dense(units=64, activation='relu')(dropout2)
  dropout3 = Dropout(0.5)(dense3)

  dense4 = Dense(units=32, activation='relu')(dropout3)
  dropout4 = Dropout(0.5)(dense4)

  output = Dense(units=1, activation='sigmoid')(dropout4)

  model = Model(inputs=[user, genre], outputs=output)
  opt = Adam(lr=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model


In [127]:
model = Recommend(n_users, n_factors)
model.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_80 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_27 (Embedding)       (None, 1, 42)        103404      ['input_80[0][0]']               
                                                                                                  
 reshape_25 (Reshape)           (None, 42)           0           ['embedding_27[0][0]']           
                                                                                                  
 input_81 (InputLayer)          [(None, 42)]         0           []                               
                                                                                           

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [128]:
X_train_array[1] = np.reshape(X_train_array[1], (-1,42))
X_train_array[0] = X_train_array[0].reshape((-1,1))

results = model.fit(x=[X_train_array[0],X_train_array[1]], y=y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test_array, y_test))



Epoch 1/100
5967/5967 [==============================] - 35s 4ms/step - loss: 0.4963 - accuracy: 0.2678 - val_loss: 0.4577 - val_accuracy: 0.2872
Epoch 2/100
5967/5967 [==============================] - 26s 4ms/step - loss: 0.4676 - accuracy: 0.2850 - val_loss: 0.4538 - val_accuracy: 0.2932
Epoch 3/100
5967/5967 [==============================] - 25s 4ms/step - loss: 0.4630 - accuracy: 0.2861 - val_loss: 0.4511 - val_accuracy: 0.2939
Epoch 4/100
5967/5967 [==============================] - 26s 4ms/step - loss: 0.4604 - accuracy: 0.2882 - val_loss: 0.4501 - val_accuracy: 0.2970
Epoch 5/100
5967/5967 [==============================] - 25s 4ms/step - loss: 0.4577 - accuracy: 0.2898 - val_loss: 0.4481 - val_accuracy: 0.2936
Epoch 6/100
5967/5967 [==============================] - 24s 4ms/step - loss: 0.4556 - accuracy: 0.2904 - val_loss: 0.4473 - val_accuracy: 0.2975
Epoch 7/100
5967/5967 [==============================] - 25s 4ms/step - loss: 0.4540 - accuracy: 0.2916 - val_loss: 0.4512 -

In [155]:
#score = model.evaluate(X_test_array, y_test, verbose=0)

# Print accuracy
print('Test accuracy:', score)
# Make predictions on test set

uid = 6

x_t = copy.deepcopy(hold[(hold['user_id'] == uid) & (hold['rating'] == -1)])
x_t = x_t.drop(['anime_id','rating'],axis=1)


x_t = [x_t['user_id'].values.tolist(), x_t.iloc[:, 2:].values.tolist()]
x_t[1] = np.reshape(x_t[1], (-1,42))
x_t[0] = np.reshape(x_t[0],(-1,1))

y_pred = model.predict(x_t)


y_pred = np.round(y_pred, 3)

# Print first 10 predictions
print(y_pred[:5])




Test accuracy: [0.46606776118278503, 0.2980809509754181]
2/2 [==============================] - 0s 3ms/step
[[0.762]
 [0.728]
 [0.775]
 [0.75 ]
 [0.773]]
